# Natural Convection with SpatialPy

## Definition of the model

### Imports and definitions

In [1]:
import os
import sys
sys.path.insert(1, "../..")
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [2]:
class Cylinder(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return x[0]**2 + x[1]**2 < 0.1**2

In [3]:
class Wall(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] < -0.5 or x[0] > 0.5: return True
        if x[1] < -0.5 or x[1] > 0.5: return True
        return False

### Model

In [4]:
class NaturalConvection(spatialpy.Model):
    def __init__(self, model_name="Natural Convection", Ra=1e4):
        spatialpy.Model.__init__(self, model_name)
        
        # System constants
        L = 1                # characteristic lenght of the cavity (= width = height)
        nxF, nyF = 200, 200  # number of fluid particles in x and y-direction
        Sc = 0.7             # Schmidt number
        nW = 3              # number of wall points
        nu = 0.01           # fluid viscosity
        rho0 = 1             # reference fluid density
        c0 = 1              # reference speed of sound (typically, 10* max velocity)
        g = 10
        beta = 1e-6
        P0  = rho0*(g*beta*L)**2    # reference pressure
        
        # Variables
        A = spatialpy.Species(name="A", diffusion_coefficient=(1 / np.sqrt(Sc * Ra)))
        self.add_species(A)
        
        # Discretization
        nxTot = nxF + 2*nW # total number of particles in x-direction (including walls)
        nyTot = nyF + 2*nW # total number of particles in y-direction (including walls)
        
        # Compute domain bounds (including the boundary)
        dx, dy = L/nxF, L/nyF
        xLim = (-0.5-(nW-1)*dx, 0.5+(nW-1)*dx)
        yLim = (-0.5-(nW-1)*dy, 0.5+(nW-1)*dy)
        
        # Compute volume and mass per particle
        vol = (xLim[1]-xLim[0])*(yLim[1]-yLim[0])*1.0 # in 2D simulations, consider z-lenght = 1
        mPP = rho0*vol/(nxTot*nyTot)                   # density * total volume / total number of particles
        
        # Domain
        self.domain = spatialpy.Domain.create_2D_domain(
            xlim=xLim, ylim=yLim, nx=nxTot, ny=nyTot,
            type_id=1, mass=mPP, nu=nu, rho0=rho0, c0=c0, P0=P0, fixed=False
        )
        
        # Types
        self.set_type(Wall(), 2, fixed=True)
        self.set_type(Cylinder(), 3, fixed=True)
        
        # Boundary Conditions
        self.add_boundary_condition(spatialpy.BoundaryCondition(
            type_id=3,
            species="A",
            value=1
        ))
        self.add_boundary_condition(spatialpy.BoundaryCondition(
            type_id=3,
            species="A",
            value=0
        ))
        
        self.timespan(np.arange(0, 10+1e-4, 0.01), timestep_size=1e-4)

In [5]:
model = NaturalConvection()

In [6]:
solver = spatialpy.Solver(model=model)
%time solver.compile(debug=True)

CPU times: user 450 ms, sys: 29 ms, total: 479 ms
Wall time: 19min 4s


In [ ]:
%time results = solver.run(debug=True)

In [ ]:
results.plot_property("v", t_ndx=4, use_matplotlib=True)

In [ ]:
t_val = results.get_timespan()[-1]
results.plot_property("v", t_val=t_val, use_matplotlib=True)

In [ ]:
results.plot_property("type", use_matplotlib=True)